# TFM - Predicción potenciales compradores de un ecommerce

Este trabajo, está principalmente enfocado a la creación de un modelo de predicción de potenciales compradores en un ecommerce real relacionado con el mundo empresarial.

Se trata de realizar una clasificación de visitantes a una web de venta online en función de los datos que se han almacenado de ellos y establecer su nivel potencial de compra.

Para ello se dispone de diferentes repositorios con datos históricos relativos a:
* características básicas del registro/lead: fecha registro, procedencia, marca de comprador, cuando compró.
* características identificativas anonimizadas de los mismos: email, ubicación geográfica, tipo de persona.
* log de sus movimientos/navegación en la web.
* información sobre los productos que ha comprado.

**Información relevante del negocio**

Los datos proceden de un ecommerce con sede en Colombia que se dedica a la venta de productos relacionados con la información de empresas del país: Informes Comerciales y módulos de información detallada sobre Datos Financieros, Prensa, Administradores, Incidencias, etc, Informes Sectoriales, Base de datos a medida, Productos de Marketing (mercadeo), Información de accionistas, Información de proveedores y clientes, etc.

Los usuarios potenciales compradores (personas en su nombre o representando a una empresa u otra entidad), llegan a la web del ecommerce por diferentes canales: directorios propios, webs de terceros, desde buscadores por labores de SEO y de SEM, etc.  

Dichos usuarios tienen acceso a diferentes productos de menor contenido a cambio de registrarse en un formulario del site, en el que cumplimentan datos personales, email, teléfono, profesión, etc. El usuario pasa a ser Registrado, y es lo que en el mundo del ecommerce se denomina “lead”.  

Tras el registro se permite el acceso a información muy básica sobre las empresas buscadas (Ficha de empresa), y se otorga la posibilidad de consumir gratis 5 productos de información a los que llamamos perfiles de empresa.  

La oferta caduca a los 30 días.  

El Perfil de empresa es un producto con contenido básico y su objetivo es mostrar a los potenciales compradores el nivel del contenido de los productos que suministra el ecommerce.  

Resumiendo, la dinámica es la siguiente:
* Un usuario busca una empresa en internet o en la web.
* Al seleccionar una de las listadas en la búsqueda, se presenta una Ficha (Identificación) y se crea un registro en el log. Se muestra los productos disponibles para consumir.  
    * Productos Promocionales (Perfil de Empresa). Para usuarios Registrados, si el Usuario se registra.
    * Resto de Productos son de Pago. Es necesario estar registrado y pagar.
* El usuario consume un producto (registro en el log) o vuelve a buscar otra empresa.  

Si el usuario Registrado está interesado en conocer más a fondo una empresa o un determinado producto de pago, se le ofrece la posibilidad de contratación:  
* PPV: Compra puntual del producto
* Bonos: compra de un conjunto de unidades o una cantidad de productos a cambio de un pago anticipado.
* Suscripción: pagando una cantidad periodica permite el acceso y consumo de productos, limitado por el volumen de compra y por la fecha de caducidad de la suscripción

Cuando se produce una de estas contrataciones el usuario “Registrado”, pasa a ser “Cliente”.

El propietario del ecommerce tiene información del usuario de su plataforma de Google Analytics que utiliza para la captación en internet, uso de cookies, estrategias de SEO y SEM, acuerdos con portales.  

Ahora desea dar un paso más y quiere conocer más sobre los usuarios registrados para determinar la probabilidad de conversión a cliente. Saber cuáles son potenciales compradores a partir de los datos de ese registro en sus sistemas, del hábito de consumo de productos, y del tipo de empresa buscada/consultada.  

El conjunto de datos está muy desbalanceado por ello es muy importante determinar a qué usuarios registrados debe dirigir el ecommerce sus campañas de captación.  

Por otra parte, pero relacionado con el mismo tema, desean saber que perfil tienen sus clientes y que variables son las mas influyentes/relevantes en la predicción de compra para modificar la estrategia de marketing y su posicionamiento en Internet.  


------------------------------------

### Librerias

In [1]:
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
# pd.options.display.max_columns

------------------------------------

### Lectura de Ficheros

A continuación comenzamos leyendo los ficheros que hemos tratado, limpiado y analizado en los procesos anteriores.

In [2]:
df_cons = pd.read_csv('Data/Consumos_Clean.txt', sep = ';', encoding = 'ISO-8859-1')

In [3]:
df_cons.shape

(367707, 29)

In [4]:
df_cons.dtypes

ID_USUARIO                             int64
1A_CONS_DESCPRODUCTO                  object
1A_CONS_DESCGRUPOPROD                 object
1A_CONS_FECHACONSUMO                  object
1A_CONS_TIPO_CONSUMO                  object
1A_CONS_EMPCONSUL_EST                 object
1A_CONS_EMPCONSUL_SECCION_DESC        object
1A_CONS_EMPCONSUL_SECTOR_ECONOMICO    object
1A_CONS_EMPCONSUL_PROV                object
PROD_CONSUM_TOT                        int64
PROMOCION_TOT                          int64
PAGO_TOT                               int64
FICH_BASIC_PROMO_TOT                   int64
FICH_AVANZADA_TOT                      int64
PERFIL_TOT                             int64
MODU_INFORM_TOT                        int64
REPORTES_TOT                           int64
OTROS_TOT                              int64
EST_ACTIVA_TOT                         int64
EST_INACTIVA_TOT                       int64
EST_REESTRUCT_TOT                      int64
EST_CLINTON_TOT                        int64
EST_NI_TOT

In [5]:
df_user = pd.read_csv('Data/Usuarios_Clean.txt', sep = ';', encoding = 'ISO-8859-1')

In [6]:
df_user.shape

(367637, 25)

In [7]:
df_user.dtypes

ID_USUARIO               int64
TIPOUSUARIO             object
FECHA_REGISTRO          object
CANAL_REGISTRO           int64
FECHA_ALTA              object
IND_ALTA                 int64
FECHA_CLIENTE           object
IND_CLIENTE              int64
TIPOEMAIL               object
BONDAD_EMAIL             int64
TIPO_TELF               object
IPCASOS                  int64
IP_COUNTRY              object
IP_REGION               object
IP_COUNTRY_PIB          object
IP_COUNTRY_IDH_NUM      object
IP_COUNTRY_IDH_GROUP    object
USU_TIPO                object
USU_TAMANIO             object
USU_ESTADO              object
USU_SECCION_DESC        object
USU_SECTOR_ECONOMICO    object
USU_DEPARTAMENTO        object
USU_DEPART_DEN_POB      object
USU_DEPART_IDH          object
dtype: object

--------------------------

### Unificamos la información

Añadimos a la iformación de cada cliente, su respectiva información de cosumo

In [8]:
df = pd.merge(left = df_user, right = df_cons, how = "inner", left_on = "ID_USUARIO", right_on = "ID_USUARIO")

In [9]:
df.head(5)

,ID_USUARIO,TIPOUSUARIO,FECHA_REGISTRO,CANAL_REGISTRO,FECHA_ALTA,IND_ALTA,FECHA_CLIENTE,IND_CLIENTE,TIPOEMAIL,BONDAD_EMAIL,TIPO_TELF,IPCASOS,IP_COUNTRY,IP_REGION,IP_COUNTRY_PIB,IP_COUNTRY_IDH_NUM,IP_COUNTRY_IDH_GROUP,USU_TIPO,USU_TAMANIO,USU_ESTADO,USU_SECCION_DESC,USU_SECTOR_ECONOMICO,USU_DEPARTAMENTO,USU_DEPART_DEN_POB,USU_DEPART_IDH,1A_CONS_DESCPRODUCTO,1A_CONS_DESCGRUPOPROD,1A_CONS_FECHACONSUMO,1A_CONS_TIPO_CONSUMO,1A_CONS_EMPCONSUL_EST,1A_CONS_EMPCONSUL_SECCION_DESC,1A_CONS_EMPCONSUL_SECTOR_ECONOMICO,1A_CONS_EMPCONSUL_PROV,PROD_CONSUM_TOT,PROMOCION_TOT,PAGO_TOT,FICH_BASIC_PROMO_TOT,FICH_AVANZADA_TOT,PERFIL_TOT,MODU_INFORM_TOT,REPORTES_TOT,OTROS_TOT,EST_ACTIVA_TOT,EST_INACTIVA_TOT,EST_REESTRUCT_TOT,EST_CLINTON_TOT,EST_NI_TOT,SECT_PRIM_TOT,SECT_SECU_TOT,SECT_TERC_TOT,SECT_CUAT_TOT,SECT_QUIN_TOT,SECT_SD_TOT
0,8107310,PF,2019-10-22,3,NoApl,0,NoApl,0,GOOHOTYAHMAIL,0,NO_PUBLICO,1,COLOMBIA,Valle del Cauca,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,PERFIL PROMOCIONAL,PERFIL,2019-10-22,Promocion,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,CAUCA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,7784565,PJ,2019-05-14,3,NoApl,0,NoApl,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Antioquia,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,PQ,ACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,QUINDIO,306.38,0.832,PERFIL PROMOCIONAL,PERFIL,2019-05-14,Promocion,ACTIVA,SD,SD,CAUCA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,7718778,PJ,2019-09-04,7,NoApl,0,NoApl,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Bolivar,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,MC,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,ATLANTICO,815.17,0.835,PERFIL PROMOCIONAL,PERFIL,2019-09-04,Promocion,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,BOGOTA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3,7952765,PF,2019-12-08,3,NoApl,0,NoApl,0,DOCENTE,1,NO_PUBLICO,1,COLOMBIA,Risaralda,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,PERFIL PROMOCIONAL,PERFIL,2019-12-08,Promocion,ACTIVA,"ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y TÉCNICAS",CUATERNARIO,RISARALDA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,7855424,PJ,2019-06-21,7,NoApl,0,NoApl,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Atlántico,Ingreso mediano alto,0.767,Alto,EMPRESARIO INDIVIDUAL,MC,INACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,ATLANTICO,815.17,0.835,PERFIL PROMOCIONAL,PERFIL,2019-06-21,Promocion,ACTIVA,INDUSTRIAS MANUFACTURERAS,SECUNDARIO,ATLANTICO,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [10]:
df.isna().sum()

ID_USUARIO                            0
TIPOUSUARIO                           0
FECHA_REGISTRO                        0
CANAL_REGISTRO                        0
FECHA_ALTA                            0
IND_ALTA                              0
FECHA_CLIENTE                         0
IND_CLIENTE                           0
TIPOEMAIL                             0
BONDAD_EMAIL                          0
TIPO_TELF                             0
IPCASOS                               0
IP_COUNTRY                            0
IP_REGION                             0
IP_COUNTRY_PIB                        0
IP_COUNTRY_IDH_NUM                    0
IP_COUNTRY_IDH_GROUP                  0
USU_TIPO                              0
USU_TAMANIO                           0
USU_ESTADO                            0
USU_SECCION_DESC                      0
USU_SECTOR_ECONOMICO                  0
USU_DEPARTAMENTO                      0
USU_DEPART_DEN_POB                    0
USU_DEPART_IDH                        0


-------------------

### Creación de nuevas variables

En este apartado trabajaremos con las variables del dataframe de forma que se tengan nuevas variables.

**1. Extración de información sobre Fechas**

Tenemos un total de hasta 4 Fechas en el dataset:
* FECHA_REGISTRO
* FECHA_ALTA
* FECHA_CLIENTE
* 1A_CONS_FECHACONSUMO

De cada variable indicada se generará la siguiente información:
- `business_day`: Si el día es laboral o no
- `weekday`: Variable categórica con el día de la semana

**Transformacion de business_day**

In [11]:
def is_business_day(date):
    try:
        return bool(len(pd.bdate_range(date, date)))
    except:
        s = 1

In [12]:
df['FECHA_REGISTRO_BUSINESS_DAY'] = df['FECHA_REGISTRO'].apply(is_business_day).fillna(-1)
df['FECHA_REGISTRO_BUSINESS_DAY'] = df['FECHA_REGISTRO_BUSINESS_DAY'].astype(int)

In [13]:
df['FECHA_ALTA_BUSINESS_DAY'] = df['FECHA_ALTA'].apply(is_business_day).fillna(-1)
df['FECHA_ALTA_BUSINESS_DAY'] = df['FECHA_ALTA_BUSINESS_DAY'].astype(int)

In [14]:
df['FECHA_CLIENTE_BUSINESS_DAY'] = df['FECHA_CLIENTE'].apply(is_business_day).fillna(-1)
df['FECHA_CLIENTE_BUSINESS_DAY'] = df['FECHA_CLIENTE_BUSINESS_DAY'].astype(int)

In [15]:
df['FECHACONSUMO_BUSINESS_DAY'] = df['1A_CONS_FECHACONSUMO'].apply(is_business_day).fillna(-1)
df['FECHACONSUMO_BUSINESS_DAY'] = df['FECHACONSUMO_BUSINESS_DAY'].astype(int)

Comprobación

In [16]:
df['FECHA_REGISTRO_BUSINESS_DAY'].value_counts()

1    298034
0     69598
Name: FECHA_REGISTRO_BUSINESS_DAY, dtype: int64

In [17]:
df['FECHA_ALTA_BUSINESS_DAY'].value_counts()

-1    363923
 1      3073
 0       636
Name: FECHA_ALTA_BUSINESS_DAY, dtype: int64

In [18]:
df['FECHA_CLIENTE_BUSINESS_DAY'].value_counts()

-1    365018
 1      2177
 0       437
Name: FECHA_CLIENTE_BUSINESS_DAY, dtype: int64

In [19]:
df['FECHACONSUMO_BUSINESS_DAY'].value_counts()

1    297996
0     69636
Name: FECHACONSUMO_BUSINESS_DAY, dtype: int64

**Transformacion de weekday**

In [20]:
df.loc[df['FECHA_REGISTRO'] == 'NoApl', 'FECHA_REGISTRO'] = None
df['FECHA_REGISTRO'] = pd.to_datetime(df['FECHA_REGISTRO'])
df['FECHA_REGISTRO_WEEKDAY'] = df['FECHA_REGISTRO'].dt.weekday
df['FECHA_REGISTRO_WEEKDAY'] = df['FECHA_REGISTRO_WEEKDAY'].fillna(-1)
df['FECHA_REGISTRO_WEEKDAY'] = df['FECHA_REGISTRO_WEEKDAY'].astype('int')

In [21]:
df['FECHA_REGISTRO_WEEKDAY'].value_counts()

2    63276
3    62309
1    60793
0    59505
4    52151
5    37780
6    31818
Name: FECHA_REGISTRO_WEEKDAY, dtype: int64

In [22]:
df.loc[df['FECHA_ALTA'] == 'NoApl', 'FECHA_ALTA'] = None
df['FECHA_ALTA'] = pd.to_datetime(df['FECHA_ALTA'])
df['FECHA_ALTA_WEEKDAY'] = df['FECHA_ALTA'].dt.weekday
df['FECHA_ALTA_WEEKDAY'] = df['FECHA_ALTA_WEEKDAY'].fillna(-1)
df['FECHA_ALTA_WEEKDAY'] = df['FECHA_ALTA_WEEKDAY'].astype('int')

In [23]:
df['FECHA_ALTA_WEEKDAY'].value_counts()

-1    363923
 2       670
 1       661
 0       621
 3       598
 4       523
 6       322
 5       314
Name: FECHA_ALTA_WEEKDAY, dtype: int64

In [24]:
df.loc[df['FECHA_CLIENTE'] == 'NoApl', 'FECHA_CLIENTE'] = None
df['FECHA_CLIENTE'] = pd.to_datetime(df['FECHA_CLIENTE'])
df['FECHA_CLIENTE_WEEKDAY'] = df['FECHA_CLIENTE'].dt.weekday
df['FECHA_CLIENTE_WEEKDAY'] = df['FECHA_CLIENTE_WEEKDAY'].fillna(-1)
df['FECHA_CLIENTE_WEEKDAY'] = df['FECHA_CLIENTE_WEEKDAY'].astype('int')

In [25]:
df['FECHA_CLIENTE_WEEKDAY'].value_counts()

-1    365018
 1       470
 2       458
 3       449
 0       432
 4       368
 5       225
 6       212
Name: FECHA_CLIENTE_WEEKDAY, dtype: int64

In [26]:
df.loc[df['1A_CONS_FECHACONSUMO'] == 'NoApl', '1A_CONS_FECHACONSUMO'] = None
df['1A_CONS_FECHACONSUMO'] = pd.to_datetime(df['1A_CONS_FECHACONSUMO'])
df['FECHACONSUMO_WEEKDAY'] = df['1A_CONS_FECHACONSUMO'].dt.weekday
df['FECHACONSUMO_WEEKDAY'] = df['FECHACONSUMO_WEEKDAY'].fillna(-1)
df['FECHACONSUMO_WEEKDAY'] = df['FECHACONSUMO_WEEKDAY'].astype('int')

In [27]:
df['FECHACONSUMO_WEEKDAY'].value_counts()

2    63237
3    62398
1    60852
0    59420
4    52089
5    37909
6    31727
Name: FECHACONSUMO_WEEKDAY, dtype: int64

**2. IP_CONTINENTE**

Se creará una variable que agrupe por continente cada uno de los paises de la variable IP_COUNTRY

In [28]:
df['IP_COUNTRY'].unique()

array(['COLOMBIA', 'NI', 'ESTADOS UNIDOS', 'SUIZA', 'PERU', 'REINO UNIDO',
       'VENEZUELA', 'CHILE', 'ECUADOR', 'JAPON', 'ARGENTINA', 'PANAMA',
       'FRANCIA', 'CAMERUN', 'ESPAÑA', 'CANADA', 'BRASIL', 'ESLOVENIA',
       'ALEMANIA', 'MEXICO', 'AUSTRIA', 'PAISES BAJOS', 'URUGUAY',
       'AUSTRALIA', 'COSTA RICA', 'SUECIA', 'ITALIA', 'FINLANDIA',
       'CHINA', 'RUSIA', 'BOLIVIA', 'GUATEMALA', 'PARAGUAY', 'POLONIA',
       'MALTA', 'EMIRATOS ARABES UNIDOS', 'SINGAPUR', 'PUERTO RICO',
       'INDIA', 'BELGICA', 'ARUBA', 'HONG KONG', 'ANDORRA',
       'REPUBLICA DOMINICANA', 'NORUEGA', 'ISRAEL', 'BELICE', 'JERSEY',
       'REPUBLICA CHECA', 'NICARAGUA', 'CURACAO', 'BONAIRE', 'TAIWAN',
       'TURQUIA', 'COREA', 'VIETNAM', 'CUBA', 'ARABIA SAUDITA', 'KUWAIT',
       'QATAR', 'GUINEA ECUATORIAL', 'NUEVA ZELANDA', 'DINAMARCA',
       'RUMANIA', 'HONDURAS', 'PORTUGAL', 'ANGOLA', 'UCRANIA', 'NIGERIA',
       'CROACIA', 'TAILANDIA', 'LUXEMBURGO', 'IRLANDA', 'GUADALUPE',
       'EL SALVADOR

In [29]:
df['IP_CONTINENTE'] = None

In [30]:
# EUROPA
df.loc[(df['IP_COUNTRY'].isin(['SUIZA','REINO UNIDO','FRANCIA','ESPAÑA','ESLOVENIA','ALEMANIA','AUSTRIA','PAISES BAJOS',
                               'SUECIA','ITALIA','FINLANDIA','RUSIA','POLONIA','MALTA','BELGICA','ANDORRA','NORUEGA',
                               'JERSEY','REPUBLICA CHECA','DINAMARCA','RUMANIA','PORTUGAL','UCRANIA','CROACIA','LUXEMBURGO', 
                               'IRLANDA','MOLDAVIA'])), 'IP_CONTINENTE'] = 'EUROPA'

# ASIA
df.loc[(df['IP_COUNTRY'].isin(['JAPON','CHINA','EMIRATOS ARABES UNIDOS','SINGAPUR','INDIA','HONG KONG','ISRAEL',
                               'TAIWAN','BAHREIN','TAILANDIA','COREA','TURQUIA','VIETNAM','ARABIA SAUDITA', 
                               'KUWAIT','QATAR'])), 'IP_CONTINENTE'] = 'ASIA'

# AFRICA
df.loc[(df['IP_COUNTRY'].isin(['CAMERUN','GUINEA ECUATORIAL','ANGOLA','NIGERIA'])), 'IP_CONTINENTE'] = 'AFRICA'

# OCEANNIA
df.loc[(df['IP_COUNTRY'].isin(['AUSTRALIA','NUEVA ZELANDA'])), 'IP_CONTINENTE'] = 'OCEANNIA'

# AMERICA DEL SUR
df.loc[(df['IP_COUNTRY'].isin(['COLOMBIA','PERU','VENEZUELA','CHILE','ECUADOR','ARGENTINA','BRASIL','URUGUAY',
                               'BOLIVIA','PARAGUAY','ARUBA','CURACAO','BONAIRE'])), 'IP_CONTINENTE'] = 'AMERICA_SUR'

# AMERICA DEL NORTE
df.loc[(df['IP_COUNTRY'].isin(['ESTADOS UNIDOS','PANAMA','CANADA','MEXICO','COSTA RICA','GUATEMALA','PUERTO RICO',
                               'REPUBLICA DOMINICANA','BELICE','NICARAGUA','BARBADOS','CUBA','HONDURAS','GUADALUPE',
                               'EL SALVADOR'])), 'IP_CONTINENTE'] = 'AMERICA_NORTE'

# NO IDENTIFICADO
df.loc[(df['IP_COUNTRY'].isin(['NI'])), 'IP_CONTINENTE'] = 'NI'

In [31]:
df['IP_CONTINENTE'].value_counts()

AMERICA_SUR      341936
NI                21759
AMERICA_NORTE      3165
EUROPA              612
ASIA                122
OCEANNIA             34
AFRICA                4
Name: IP_CONTINENTE, dtype: int64

In [32]:
df['IP_CONTINENTE'].unique()

array(['AMERICA_SUR', 'NI', 'AMERICA_NORTE', 'EUROPA', 'ASIA', 'AFRICA',
       'OCEANNIA'], dtype=object)

**3. IP_COLOMBIA**

La siguiente variable nos informa de si la ip del registro se realiza desde colombia (mismo lugar perteneciente al ecommerce) o no
* 1: La IP es desde Colombia
* 0: Caso Contrario

In [33]:
df['IP_COLOMBIA'] = None

In [34]:
df.loc[(df['IP_COUNTRY'].isin(['COLOMBIA'])), 'IP_COLOMBIA'] = 1
df.loc[(~df['IP_COUNTRY'].isin(['COLOMBIA'])), 'IP_COLOMBIA'] = 0

In [35]:
df['IP_COLOMBIA'].value_counts()

1    340300
0     27332
Name: IP_COLOMBIA, dtype: int64

**4. EMAIL_PERSONAL**

Con esta variable se pretende crear un indicador que informe si el correo electrónico es personal o no

In [36]:
df['TIPOEMAIL'].unique()

array(['GOOHOTYAHMAIL', 'DOCENTE', 'CORPORATIVO', 'TEMPORAL',
       'SINCLASIFICAR', 'ADMINISTRACION'], dtype=object)

In [37]:
df['TIPOEMAIL'].value_counts()

GOOHOTYAHMAIL     289526
CORPORATIVO        33859
SINCLASIFICAR      20502
DOCENTE            16362
TEMPORAL            6943
ADMINISTRACION       440
Name: TIPOEMAIL, dtype: int64

In [38]:
df['EMAIL_PERSONAL'] = None

df.loc[(df['TIPOEMAIL'].isin(['GOOHOTYAHMAIL'])), 'EMAIL_PERSONAL'] = 1
df.loc[(~df['TIPOEMAIL'].isin(['GOOHOTYAHMAIL'])), 'EMAIL_PERSONAL'] = 0

In [39]:
df['EMAIL_PERSONAL'].value_counts()

1    289526
0     78106
Name: EMAIL_PERSONAL, dtype: int64

**5. Indicador de fechas**

Realizaremos indicadores de coincidencia de fechas

**5.1** Fecha de registro == Fecha de Alta (cliente)

* 1: Si
* 0: No
* -1: No aplica

In [40]:
df['REGISTRO_ALTA'] = (df['FECHA_REGISTRO'] == df['FECHA_ALTA'])
df['REGISTRO_ALTA'] = df['REGISTRO_ALTA'].apply(lambda x: int(x))
df.loc[(df['FECHA_ALTA'].isna()), 'REGISTRO_ALTA'] = -1

In [41]:
df['REGISTRO_ALTA'].value_counts()

-1    363923
 1      2345
 0      1364
Name: REGISTRO_ALTA, dtype: int64

In [42]:
df['ALTA_CLIENTE'] = (df['FECHA_ALTA'] == df['FECHA_CLIENTE'])
df['ALTA_CLIENTE'] = df['ALTA_CLIENTE'].apply(lambda x: int(x))
df.loc[(df['FECHA_CLIENTE'].isna()), 'ALTA_CLIENTE'] = -1

In [43]:
df['ALTA_CLIENTE'].value_counts()

-1    365018
 1      2430
 0       184
Name: ALTA_CLIENTE, dtype: int64

In [44]:
df['CLIENTE_CONSUMO'] = (df['FECHA_CLIENTE'] == df['1A_CONS_FECHACONSUMO'])
df['CLIENTE_CONSUMO'] = df['CLIENTE_CONSUMO'].apply(lambda x: int(x))
df.loc[(df['FECHA_CLIENTE'].isna()), 'CLIENTE_CONSUMO'] = -1

In [45]:
df['CLIENTE_CONSUMO'].value_counts()

-1    365018
 0      1349
 1      1265
Name: CLIENTE_CONSUMO, dtype: int64

**6. meses de las fechas**

In [46]:
df.loc[df['FECHA_REGISTRO'] == 'NoApl', 'FECHA_REGISTRO'] = None
df['FECHA_REGISTRO'] = pd.to_datetime(df['FECHA_REGISTRO'])
df['FECHA_REGISTRO_MONTH'] = df['FECHA_REGISTRO'].dt.month
df['FECHA_REGISTRO_MONTH'] = df['FECHA_REGISTRO_MONTH'].fillna(-1)
df['FECHA_REGISTRO_MONTH'] = df['FECHA_REGISTRO_MONTH'].astype('int')

In [47]:
df['FECHA_REGISTRO_MONTH'].value_counts()

5     37687
8     32403
11    32106
10    31948
7     31585
4     31532
3     31481
9     30927
2     29900
6     28172
1     25343
12    24548
Name: FECHA_REGISTRO_MONTH, dtype: int64

In [48]:
df.loc[df['FECHA_ALTA'] == 'NoApl', 'FECHA_ALTA'] = None
df['FECHA_ALTA'] = pd.to_datetime(df['FECHA_ALTA'])
df['FECHA_ALTA_MONTH'] = df['FECHA_ALTA'].dt.month
df['FECHA_ALTA_MONTH'] = df['FECHA_ALTA_MONTH'].fillna(-1)
df['FECHA_ALTA_MONTH'] = df['FECHA_ALTA_MONTH'].astype('int')

In [49]:
df['FECHA_ALTA_MONTH'].value_counts()

-1     363923
 11       374
 7        369
 8        340
 10       338
 5        337
 9        328
 4        297
 6        278
 1        275
 2        265
 3        255
 12       253
Name: FECHA_ALTA_MONTH, dtype: int64

In [50]:
df.loc[df['FECHA_CLIENTE'] == 'NoApl', 'FECHA_CLIENTE'] = None
df['FECHA_CLIENTE'] = pd.to_datetime(df['FECHA_CLIENTE'])
df['FECHA_CLIENTE_MONTH'] = df['FECHA_CLIENTE'].dt.month
df['FECHA_CLIENTE_MONTH'] = df['FECHA_CLIENTE_MONTH'].fillna(-1)
df['FECHA_CLIENTE_MONTH'] = df['FECHA_CLIENTE_MONTH'].astype('int')

In [51]:
df['FECHA_CLIENTE_MONTH'].value_counts()

-1     365018
 11       260
 7        255
 8        244
 9        243
 10       239
 5        232
 6        200
 2        199
 1        197
 4        187
 12       187
 3        171
Name: FECHA_CLIENTE_MONTH, dtype: int64

In [52]:
df.loc[df['1A_CONS_FECHACONSUMO'] == 'NoApl', '1A_CONS_FECHACONSUMO'] = None
df['1A_CONS_FECHACONSUMO'] = pd.to_datetime(df['1A_CONS_FECHACONSUMO'])
df['FECHACONSUMO_MONTH'] = df['1A_CONS_FECHACONSUMO'].dt.month
df['FECHACONSUMO_MONTH'] = df['FECHACONSUMO_MONTH'].fillna(-1)
df['FECHACONSUMO_MONTH'] = df['FECHACONSUMO_MONTH'].astype('int')

In [53]:
df['FECHACONSUMO_MONTH'].value_counts()

5     37886
8     32161
4     31837
3     31819
11    31636
10    31568
7     31472
9     30630
2     30446
6     28258
1     25989
12    23930
Name: FECHACONSUMO_MONTH, dtype: int64

------------

------------

In [54]:
df.head()

,ID_USUARIO,TIPOUSUARIO,FECHA_REGISTRO,CANAL_REGISTRO,FECHA_ALTA,IND_ALTA,FECHA_CLIENTE,IND_CLIENTE,TIPOEMAIL,BONDAD_EMAIL,TIPO_TELF,IPCASOS,IP_COUNTRY,IP_REGION,IP_COUNTRY_PIB,IP_COUNTRY_IDH_NUM,IP_COUNTRY_IDH_GROUP,USU_TIPO,USU_TAMANIO,USU_ESTADO,USU_SECCION_DESC,USU_SECTOR_ECONOMICO,USU_DEPARTAMENTO,USU_DEPART_DEN_POB,USU_DEPART_IDH,1A_CONS_DESCPRODUCTO,1A_CONS_DESCGRUPOPROD,1A_CONS_FECHACONSUMO,1A_CONS_TIPO_CONSUMO,1A_CONS_EMPCONSUL_EST,1A_CONS_EMPCONSUL_SECCION_DESC,1A_CONS_EMPCONSUL_SECTOR_ECONOMICO,1A_CONS_EMPCONSUL_PROV,PROD_CONSUM_TOT,PROMOCION_TOT,PAGO_TOT,FICH_BASIC_PROMO_TOT,FICH_AVANZADA_TOT,PERFIL_TOT,MODU_INFORM_TOT,REPORTES_TOT,OTROS_TOT,EST_ACTIVA_TOT,EST_INACTIVA_TOT,EST_REESTRUCT_TOT,EST_CLINTON_TOT,EST_NI_TOT,SECT_PRIM_TOT,SECT_SECU_TOT,SECT_TERC_TOT,SECT_CUAT_TOT,SECT_QUIN_TOT,SECT_SD_TOT,FECHA_REGISTRO_BUSINESS_DAY,FECHA_ALTA_BUSINESS_DAY,FECHA_CLIENTE_BUSINESS_DAY,FECHACONSUMO_BUSINESS_DAY,FECHA_REGISTRO_WEEKDAY,FECHA_ALTA_WEEKDAY,FECHA_CLIENTE_WEEKDAY,FECHACONSUMO_WEEKDAY,IP_CONTINENTE,IP_COLOMBIA,EMAIL_PERSONAL,REGISTRO_ALTA,ALTA_CLIENTE,CLIENTE_CONSUMO,FECHA_REGISTRO_MONTH,FECHA_ALTA_MONTH,FECHA_CLIENTE_MONTH,FECHACONSUMO_MONTH
0,8107310,PF,2019-10-22,3,NaT,0,NaT,0,GOOHOTYAHMAIL,0,NO_PUBLICO,1,COLOMBIA,Valle del Cauca,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,PERFIL PROMOCIONAL,PERFIL,2019-10-22,Promocion,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,CAUCA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,-1,-1,1,1,-1,-1,1,AMERICA_SUR,1,1,-1,-1,-1,10,-1,-1,10
1,7784565,PJ,2019-05-14,3,NaT,0,NaT,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Antioquia,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,PQ,ACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,QUINDIO,306.38,0.832,PERFIL PROMOCIONAL,PERFIL,2019-05-14,Promocion,ACTIVA,SD,SD,CAUCA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,-1,-1,1,1,-1,-1,1,AMERICA_SUR,1,1,-1,-1,-1,5,-1,-1,5
2,7718778,PJ,2019-09-04,7,NaT,0,NaT,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Bolivar,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,MC,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,ATLANTICO,815.17,0.835,PERFIL PROMOCIONAL,PERFIL,2019-09-04,Promocion,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,BOGOTA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,-1,-1,1,2,-1,-1,2,AMERICA_SUR,1,1,-1,-1,-1,9,-1,-1,9
3,7952765,PF,2019-12-08,3,NaT,0,NaT,0,DOCENTE,1,NO_PUBLICO,1,COLOMBIA,Risaralda,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,PERFIL PROMOCIONAL,PERFIL,2019-12-08,Promocion,ACTIVA,"ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y TÉCNICAS",CUATERNARIO,RISARALDA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,-1,-1,0,6,-1,-1,6,AMERICA_SUR,1,0,-1,-1,-1,12,-1,-1,12
4,7855424,PJ,2019-06-21,7,NaT,0,NaT,0,GOOHOTYAHMAIL,1,NO_PUBLICO,1,COLOMBIA,Atlántico,Ingreso mediano alto,0.767,Alto,EMPRESARIO INDIVIDUAL,MC,INACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,ATLANTICO,815.17,0.835,PERFIL PROMOCIONAL,PERFIL,2019-06-21,Promocion,ACTIVA,INDUSTRIAS MANUFACTURERAS,SECUNDARIO,ATLANTICO,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,-1,-1,1,4,-1,-1,4,AMERICA_SUR,1,1,-1,-1,-1,6,-1,-1,6


### Reducción de dimensionalidad

Por último eliminamos columnas, renombramos y ordenamos al gusto

In [55]:
df = df[['TIPOUSUARIO','FECHA_REGISTRO_BUSINESS_DAY','FECHA_REGISTRO_WEEKDAY','FECHA_REGISTRO_MONTH','CANAL_REGISTRO','FECHA_ALTA_BUSINESS_DAY','FECHA_ALTA_WEEKDAY','FECHA_ALTA_MONTH',
         'REGISTRO_ALTA','IND_ALTA','FECHA_CLIENTE_BUSINESS_DAY','FECHA_CLIENTE_WEEKDAY','FECHA_CLIENTE_MONTH','ALTA_CLIENTE','IND_CLIENTE','TIPOEMAIL','EMAIL_PERSONAL','BONDAD_EMAIL',
         'TIPO_TELF','IPCASOS','IP_COUNTRY','IP_CONTINENTE','IP_COLOMBIA','IP_REGION','IP_COUNTRY_PIB','IP_COUNTRY_IDH_NUM','IP_COUNTRY_IDH_GROUP','USU_TIPO','USU_TAMANIO','USU_ESTADO',
         'USU_SECCION_DESC','USU_SECTOR_ECONOMICO','USU_DEPARTAMENTO','USU_DEPART_DEN_POB','USU_DEPART_IDH','1A_CONS_DESCPRODUCTO','1A_CONS_DESCGRUPOPROD','FECHACONSUMO_BUSINESS_DAY',
         'FECHACONSUMO_WEEKDAY','FECHACONSUMO_MONTH','CLIENTE_CONSUMO','1A_CONS_TIPO_CONSUMO','1A_CONS_EMPCONSUL_EST','1A_CONS_EMPCONSUL_SECCION_DESC','1A_CONS_EMPCONSUL_SECTOR_ECONOMICO',
         '1A_CONS_EMPCONSUL_PROV','PROD_CONSUM_TOT','PROMOCION_TOT','PAGO_TOT','FICH_BASIC_PROMO_TOT','FICH_AVANZADA_TOT','PERFIL_TOT','MODU_INFORM_TOT','REPORTES_TOT','OTROS_TOT',
         'EST_ACTIVA_TOT','EST_INACTIVA_TOT','EST_REESTRUCT_TOT','EST_CLINTON_TOT','EST_NI_TOT','SECT_PRIM_TOT','SECT_SECU_TOT','SECT_TERC_TOT','SECT_CUAT_TOT','SECT_QUIN_TOT',
         'SECT_SD_TOT']]

In [56]:
df.head(3)

,TIPOUSUARIO,FECHA_REGISTRO_BUSINESS_DAY,FECHA_REGISTRO_WEEKDAY,FECHA_REGISTRO_MONTH,CANAL_REGISTRO,FECHA_ALTA_BUSINESS_DAY,FECHA_ALTA_WEEKDAY,FECHA_ALTA_MONTH,REGISTRO_ALTA,IND_ALTA,FECHA_CLIENTE_BUSINESS_DAY,FECHA_CLIENTE_WEEKDAY,FECHA_CLIENTE_MONTH,ALTA_CLIENTE,IND_CLIENTE,TIPOEMAIL,EMAIL_PERSONAL,BONDAD_EMAIL,TIPO_TELF,IPCASOS,IP_COUNTRY,IP_CONTINENTE,IP_COLOMBIA,IP_REGION,IP_COUNTRY_PIB,IP_COUNTRY_IDH_NUM,IP_COUNTRY_IDH_GROUP,USU_TIPO,USU_TAMANIO,USU_ESTADO,USU_SECCION_DESC,USU_SECTOR_ECONOMICO,USU_DEPARTAMENTO,USU_DEPART_DEN_POB,USU_DEPART_IDH,1A_CONS_DESCPRODUCTO,1A_CONS_DESCGRUPOPROD,FECHACONSUMO_BUSINESS_DAY,FECHACONSUMO_WEEKDAY,FECHACONSUMO_MONTH,CLIENTE_CONSUMO,1A_CONS_TIPO_CONSUMO,1A_CONS_EMPCONSUL_EST,1A_CONS_EMPCONSUL_SECCION_DESC,1A_CONS_EMPCONSUL_SECTOR_ECONOMICO,1A_CONS_EMPCONSUL_PROV,PROD_CONSUM_TOT,PROMOCION_TOT,PAGO_TOT,FICH_BASIC_PROMO_TOT,FICH_AVANZADA_TOT,PERFIL_TOT,MODU_INFORM_TOT,REPORTES_TOT,OTROS_TOT,EST_ACTIVA_TOT,EST_INACTIVA_TOT,EST_REESTRUCT_TOT,EST_CLINTON_TOT,EST_NI_TOT,SECT_PRIM_TOT,SECT_SECU_TOT,SECT_TERC_TOT,SECT_CUAT_TOT,SECT_QUIN_TOT,SECT_SD_TOT
0,PF,1,1,10,3,-1,-1,-1,-1,0,-1,-1,-1,-1,0,GOOHOTYAHMAIL,1,0,NO_PUBLICO,1,COLOMBIA,AMERICA_SUR,1,Valle del Cauca,Ingreso mediano alto,0.767,Alto,NoApl,NoApl,NoApl,NoApl,NoApl,NOAPL,NOAPL,NOAPL,PERFIL PROMOCIONAL,PERFIL,1,1,10,-1,Promocion,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,CAUCA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,PJ,1,1,5,3,-1,-1,-1,-1,0,-1,-1,-1,-1,0,GOOHOTYAHMAIL,1,1,NO_PUBLICO,1,COLOMBIA,AMERICA_SUR,1,Antioquia,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,PQ,ACTIVA,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,TERCIARIO,QUINDIO,306.38,0.832,PERFIL PROMOCIONAL,PERFIL,1,1,5,-1,Promocion,ACTIVA,SD,SD,CAUCA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,PJ,1,2,9,7,-1,-1,-1,-1,0,-1,-1,-1,-1,0,GOOHOTYAHMAIL,1,1,NO_PUBLICO,1,COLOMBIA,AMERICA_SUR,1,Bolivar,Ingreso mediano alto,0.767,Alto,SOC_COMERCIAL_INDUSTRIAL,MC,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,ATLANTICO,815.17,0.835,PERFIL PROMOCIONAL,PERFIL,1,2,9,-1,Promocion,ACTIVA,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,TERCIARIO,BOGOTA,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [57]:
df.shape

(367632, 66)

---------------

---------------

In [58]:
df.isna().sum()

TIPOUSUARIO                           0
FECHA_REGISTRO_BUSINESS_DAY           0
FECHA_REGISTRO_WEEKDAY                0
FECHA_REGISTRO_MONTH                  0
CANAL_REGISTRO                        0
FECHA_ALTA_BUSINESS_DAY               0
FECHA_ALTA_WEEKDAY                    0
FECHA_ALTA_MONTH                      0
REGISTRO_ALTA                         0
IND_ALTA                              0
FECHA_CLIENTE_BUSINESS_DAY            0
FECHA_CLIENTE_WEEKDAY                 0
FECHA_CLIENTE_MONTH                   0
ALTA_CLIENTE                          0
IND_CLIENTE                           0
TIPOEMAIL                             0
EMAIL_PERSONAL                        0
BONDAD_EMAIL                          0
TIPO_TELF                             0
IPCASOS                               0
IP_COUNTRY                            0
IP_CONTINENTE                         0
IP_COLOMBIA                           0
IP_REGION                             0
IP_COUNTRY_PIB                        0


### Escritura de Datos

In [59]:
df.to_csv('Data/dataset_model.txt', sep = ';', encoding = 'ISO-8859-1', index = False)